In [1]:
import pandas as pd

In [2]:
train_features = pd.read_csv('data/training_set_features.csv')
train_labels = pd.read_csv('data/training_set_labels.csv')
test_features = pd.read_csv('data/test_set_features.csv')

In [3]:
train_data = pd.merge(train_features, train_labels, on='respondent_id').set_index('respondent_id')
train_data.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0


In [32]:
# Separate X and y
target = 'seasonal_vaccine'

X = train_data.drop(columns=[target])
y = train_data[target]

# Drop features with high null count
threshold = .3
missing_data_pct = X.isnull().sum() / len(X)
high_null_features = missing_data_pct[missing_data_pct > threshold]
X = X.drop(columns=high_null_features.index)

# Encode categorical features
categorical_features = [f for f in X.columns if X[f].dtype == 'object']
X_encoded = pd.get_dummies(X, columns=categorical_features)

# Forward fill nulls in X.
X_encoded = X_encoded.ffill()

# Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=.4, random_state=42)
for name, dataset in zip(['X_train', 'X_test', 'y_train', 'y_test'], [X_train, X_test, y_train, y_test]):
    print(f'{name}: {dataset.shape}')

X_train: (16024, 61)
X_test: (10683, 61)
y_train: (16024,)
y_test: (10683,)


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

baseline_model = LogisticRegression(random_state=42, max_iter=1000)

baseline_model.fit(X_train, y_train)
baseline_pred = baseline_model.predict(X_test)

baseline_F1 = f1_score(y_test, baseline_pred)
baseline_accuracy = accuracy_score(y_test, baseline_pred)

print(f'baseline F1 score: {baseline_F1:,.4f}')
print(f'baseline accuracy: {baseline_accuracy:,.4f}')

baseline F1 score: 0.7780
baseline accuracy: 0.8000


In [34]:
train_labels

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0
...,...,...,...
26702,26702,0,0
26703,26703,0,0
26704,26704,0,1
26705,26705,0,0
